<a href="https://colab.research.google.com/github/AIVoiceAskDeeper/work/blob/main/%E3%80%90SIGNATE%E7%AC%AC36%E5%9B%9E%E3%82%B3%E3%83%B3%E3%83%9A%E3%80%91%E5%BE%93%E6%A5%AD%E5%93%A1%E3%81%AE%E9%9B%A2%E8%81%B7%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#!pip install lightgbm
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics


In [ ]:
sample = pd.read_csv("sample_submit.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
combine = [train, test]
# データの中身を確認
train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,Travel_Rarely,450.941476,Research & Development,7.601074,3,Medical,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,Travel_Rarely,730.235896,Research & Development,26.739489,3,Technical Degree,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,Travel_Rarely,1082.560066,Human Resources,7.374739,3,Life Sciences,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,Travel_Rarely,706.247579,Research & Development,14.791373,1,Medical,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,Travel_Rarely,500.610860,Research & Development,2.146966,4,Medical,981,2,...,80,1,8,3,3,1,0,0,0,0


In [ ]:
# 欠損値の確認
#train.isnull().sum()

In [ ]:
# 統計の確認
train_describe = train.describe()
train_describe

,id,Age,DailyRate,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,...,1200.0,1200.000000,1200.000000,1200.000000,1200.00000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,1013.075833,32.701667,750.754157,10.218387,2.853333,1077.855000,2.655833,66.261667,2.716667,2.125833,...,80.0,0.863333,10.631667,2.755000,2.70000,6.351667,4.249167,2.163333,3.780000,0.177500
std,575.726036,8.282199,415.420917,8.134144,1.060390,580.185495,1.139502,20.257801,0.675847,1.185821,...,0.0,0.874810,7.721335,1.256167,0.70267,5.120532,3.688507,3.312219,3.314225,0.382251
min,0.000000,17.000000,59.231581,-0.023999,1.000000,12.000000,1.000000,30.000000,1.000000,1.000000,...,80.0,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,508.750000,26.000000,403.946043,2.688173,2.000000,699.000000,2.000000,48.000000,2.000000,1.000000,...,80.0,0.000000,5.000000,2.000000,2.00000,2.000000,2.000000,0.000000,2.000000,0.000000
50%,1018.000000,34.000000,736.873463,9.345924,3.000000,1059.000000,3.000000,67.000000,3.000000,2.000000,...,80.0,1.000000,9.000000,3.000000,3.00000,5.000000,3.000000,1.000000,3.000000,0.000000
75%,1519.250000,37.000000,1092.443495,15.231438,4.000000,1591.000000,4.000000,84.000000,3.000000,3.000000,...,80.0,1.000000,15.000000,3.000000,3.00000,9.000000,7.000000,2.000000,7.000000,0.000000
max,1998.000000,56.000000,1484.979305,29.890208,5.000000,2060.000000,4.000000,100.000000,4.000000,5.000000,...,80.0,3.000000,36.000000,6.000000,4.00000,32.000000,17.000000,15.000000,17.000000,1.000000


In [ ]:
#!pip install pandas-profiling

In [ ]:

#import pandas as pd
#import pandas_profiling as pdp

#profile = pdp.ProfileReport(train)
#profile

In [ ]:
# 型の確認
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        1200 non-null   int64  
 1   Age                       1200 non-null   int64  
 2   BusinessTravel            1200 non-null   object 
 3   DailyRate                 1200 non-null   float64
 4   Department                1200 non-null   object 
 5   DistanceFromHome          1200 non-null   float64
 6   Education                 1200 non-null   int64  
 7   EducationField            1200 non-null   object 
 8   EmployeeNumber            1200 non-null   int64  
 9   EnvironmentSatisfaction   1200 non-null   int64  
 10  Gender                    1200 non-null   object 
 11  HourlyRate                1200 non-null   int64  
 12  JobInvolvement            1200 non-null   int64  
 13  JobLevel                  1200 non-null   int64  
 14  JobRole 

In [ ]:
train.loc[:, "id":"HourlyRate"].head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate
0,0,26,Travel_Rarely,450.941476,Research & Development,7.601074,3,Medical,1291,4,Male,43
1,3,47,Travel_Rarely,730.235896,Research & Development,26.739489,3,Technical Degree,1587,2,Male,98
2,7,26,Travel_Rarely,1082.560066,Human Resources,7.374739,3,Life Sciences,1591,2,Male,84
3,10,46,Travel_Rarely,706.247579,Research & Development,14.791373,1,Medical,1572,1,Female,79
4,11,25,Travel_Rarely,500.610860,Research & Development,2.146966,4,Medical,981,2,Male,98


In [ ]:
#train_corr=train.corr()
#train_corr

In [ ]:
train.describe(include=['O'])

,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
count,1200,1200,1200,1200,1200,1200,1200,1200
unique,3,3,6,2,9,3,1,2
top,Travel_Rarely,Research & Development,Life Sciences,Male,Research Scientist,Married,Y,No
freq,808,773,501,713,252,578,1200,882


In [ ]:
#train[["EducationField", "Attrition"]].groupby(['EducationField'], as_index=False).mean().sort_values(by='Attrition', ascending=False)

In [ ]:
for dataset in combine:
    dataset['BusinessTravel'] = dataset['BusinessTravel'].map( {'Non-Travel': 1,'Travel_Frequently': 2,'Travel_Rarely': 3} ).astype(int)

train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,3,450.941476,Research & Development,7.601074,3,Medical,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,3,730.235896,Research & Development,26.739489,3,Technical Degree,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,3,1082.560066,Human Resources,7.374739,3,Life Sciences,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,3,706.247579,Research & Development,14.791373,1,Medical,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,3,500.610860,Research & Development,2.146966,4,Medical,981,2,...,80,1,8,3,3,1,0,0,0,0


In [ ]:
for dataset in combine:
    dataset['Department'] = dataset['Department'].map( {'Human Resources': 1,'Research & Development': 2,'Sales': 3} ).astype(int)

train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,3,450.941476,2,7.601074,3,Medical,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,3,730.235896,2,26.739489,3,Technical Degree,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,3,1082.560066,1,7.374739,3,Life Sciences,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,3,706.247579,2,14.791373,1,Medical,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,3,500.610860,2,2.146966,4,Medical,981,2,...,80,1,8,3,3,1,0,0,0,0


In [ ]:
for dataset in combine:
    dataset['EducationField'] = dataset['EducationField'].map( {'Human Resources': 1,'Life Sciences': 2,'Marketing': 3,'Medical':4,'Other':5,'Technical Degree':6} ).astype(int)

train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,3,450.941476,2,7.601074,3,4,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,3,730.235896,2,26.739489,3,6,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,3,1082.560066,1,7.374739,3,2,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,3,706.247579,2,14.791373,1,4,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,3,500.610860,2,2.146966,4,4,981,2,...,80,1,8,3,3,1,0,0,0,0


In [ ]:
for dataset in combine:
    dataset['Gender'] = dataset['Gender'].map( {'Male': 2, 'Female': 1} ).astype(int)

train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,3,450.941476,2,7.601074,3,4,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,3,730.235896,2,26.739489,3,6,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,3,1082.560066,1,7.374739,3,2,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,3,706.247579,2,14.791373,1,4,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,3,500.610860,2,2.146966,4,4,981,2,...,80,1,8,3,3,1,0,0,0,0


In [ ]:
for dataset in combine:
    dataset['OverTime'] = dataset['OverTime'].map( {'No': 1, 'Yes': 2} ).astype(int)

train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,3,450.941476,2,7.601074,3,4,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,3,730.235896,2,26.739489,3,6,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,3,1082.560066,1,7.374739,3,2,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,3,706.247579,2,14.791373,1,4,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,3,500.610860,2,2.146966,4,4,981,2,...,80,1,8,3,3,1,0,0,0,0


In [ ]:
train.describe(include=['O'])

,JobRole,MaritalStatus,Over18
count,1200,1200,1200
unique,9,3,1
top,Research Scientist,Married,Y
freq,252,578,1200


In [ ]:
for dataset in combine:
    dataset['MaritalStatus'] = dataset['MaritalStatus'].map( {'Divorced': 1, 'Married': 2,'Single':3} ).astype(int)

train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,3,450.941476,2,7.601074,3,4,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,3,730.235896,2,26.739489,3,6,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,3,1082.560066,1,7.374739,3,2,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,3,706.247579,2,14.791373,1,4,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,3,500.610860,2,2.146966,4,4,981,2,...,80,1,8,3,3,1,0,0,0,0


In [ ]:
train[["JobRole", "Attrition"]].groupby(['JobRole'], as_index=False).mean().sort_values(by='Attrition', ascending=False)

,JobRole,Attrition
8,Sales Representative,0.425743
2,Laboratory Technician,0.256983
6,Research Scientist,0.186508
1,Human Resources,0.181818
7,Sales Executive,0.164659
3,Manager,0.133333
4,Manufacturing Director,0.085271
0,Healthcare Representative,0.054348
5,Research Director,0.044444


In [ ]:
for dataset in combine:
    dataset['JobRole'] = dataset['JobRole'].map( {'Research Director': 1, 'Healthcare Representative': 2,'Manufacturing Director':3,
                                                  'Manager': 4, 'Sales Executive': 5,'Human Resources':6,
                                                  'Research Scientist': 7, 'Laboratory Technician': 8,'Sales Representative':9} ).astype(int)

train.head()

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,26,3,450.941476,2,7.601074,3,4,1291,4,...,80,0,1,3,2,11,8,7,0,0
1,3,47,3,730.235896,2,26.739489,3,6,1587,2,...,80,1,20,1,1,5,9,4,4,0
2,7,26,3,1082.560066,1,7.374739,3,2,1591,2,...,80,1,4,3,3,3,2,0,2,0
3,10,46,3,706.247579,2,14.791373,1,4,1572,1,...,80,1,6,0,1,5,4,7,7,1
4,11,25,3,500.610860,2,2.146966,4,4,981,2,...,80,1,8,3,3,1,0,0,0,0


In [ ]:
# 型の確認
#train.info()

In [ ]:
#profile2 = pdp.ProfileReport(train)
#profile2

In [ ]:
train = train.drop(["Over18"], axis=1)
test = test.drop(["Over18"], axis=1)
combine = [train, test]

In [ ]:
# 相関の強かった2変数を掛ける
for dataset in combine:
    dataset['Department*JobRole'] = dataset.Department * dataset.JobRole

train.loc[:, ['Department*JobRole', 'Department', 'JobRole']].head(10)

,Department*JobRole,Department,JobRole
0,16,2,8
1,14,2,7
2,6,1,6
3,16,2,8
4,6,2,3
5,7,1,7
6,14,2,7
7,10,2,5
8,15,3,5
9,27,3,9


In [ ]:
# 相関の強かった2変数を掛ける
for dataset in combine:
    dataset['PercentSalaryHike*PerformanceRating'] = dataset.PercentSalaryHike * dataset.PerformanceRating

train.loc[:, ['PercentSalaryHike*PerformanceRating', 'PercentSalaryHike', 'PerformanceRating']].head(10)


,PercentSalaryHike*PerformanceRating,PercentSalaryHike,PerformanceRating
0,39,13,3
1,33,11,3
2,54,18,3
3,80,20,4
4,42,14,3
5,48,16,3
6,33,11,3
7,80,20,4
8,88,22,4
9,72,24,3


In [ ]:
'''
# qcut を用いて、DailyRateを4分割する DailyRate_area を作成
train["DailyRate_area"] =  pd.qcut(train["DailyRate"], 4)
train[["DailyRate_area", "Attrition"]].groupby(["DailyRate_area"], as_index=False).mean().sort_values(by="DailyRate_area", ascending=True)
'''

'\n# qcut を用いて、DailyRateを4分割する DailyRate_area を作成\ntrain["DailyRate_area"] =  pd.qcut(train["DailyRate"], 4)\ntrain[["DailyRate_area", "Attrition"]].groupby(["DailyRate_area"], as_index=False).mean().sort_values(by="DailyRate_area", ascending=True)\n'

In [ ]:
'''
# 作成された範囲に沿ってラベル付け
for dataset in combine:
    dataset.loc[ dataset["DailyRate"] <= 403.946, "DailyRate"] = 4
    dataset.loc[(dataset["DailyRate"] > 403.946) & (dataset["DailyRate"] <= 736.873),
                "DailyRate"] = 3
    dataset.loc[(dataset["DailyRate"] > 736.873) & (dataset["DailyRate"] <= 1092.443),
                "DailyRate"]   = 2
    dataset.loc[ dataset["DailyRate"] > 1092.443, "DailyRate"] = 1
    dataset["DailyRate"] = dataset["DailyRate"].astype(int)

train = train.drop(["DailyRate_area"], axis=1)
combine = [train, test]
'''

'\n# 作成された範囲に沿ってラベル付け\nfor dataset in combine:\n    dataset.loc[ dataset["DailyRate"] <= 403.946, "DailyRate"] = 4\n    dataset.loc[(dataset["DailyRate"] > 403.946) & (dataset["DailyRate"] <= 736.873),\n                "DailyRate"] = 3\n    dataset.loc[(dataset["DailyRate"] > 736.873) & (dataset["DailyRate"] <= 1092.443),\n                "DailyRate"]   = 2\n    dataset.loc[ dataset["DailyRate"] > 1092.443, "DailyRate"] = 1\n    dataset["DailyRate"] = dataset["DailyRate"].astype(int)\n\ntrain = train.drop(["DailyRate_area"], axis=1)\ncombine = [train, test]\n'

In [ ]:
'''
# qcut を用いて、DistanceFromHomeを4分割する DistanceFromHome_area を作成
train["DistanceFromHome_area"] =  pd.qcut(train["DistanceFromHome"], 4)
train[["DistanceFromHome_area", "Attrition"]].groupby(["DistanceFromHome_area"], as_index=False).mean().sort_values(by="DistanceFromHome_area", ascending=True)
'''

'\n# qcut を用いて、DistanceFromHomeを4分割する DistanceFromHome_area を作成\ntrain["DistanceFromHome_area"] =  pd.qcut(train["DistanceFromHome"], 4)\ntrain[["DistanceFromHome_area", "Attrition"]].groupby(["DistanceFromHome_area"], as_index=False).mean().sort_values(by="DistanceFromHome_area", ascending=True)\n'

In [ ]:
'''
# 作成された範囲に沿ってラベル付け
for dataset in combine:
    dataset.loc[ dataset["DistanceFromHome"] <= 2.688, "DistanceFromHome"] = 1
    dataset.loc[(dataset["DistanceFromHome"] > 2.688) & (dataset["DistanceFromHome"] <= 9.346),
                "DistanceFromHome"] = 2
    dataset.loc[(dataset["DistanceFromHome"] > 9.346) & (dataset["DistanceFromHome"] <= 15.231),
                "DistanceFromHome"]   = 3
    dataset.loc[ dataset["DistanceFromHome"] > 15.231, "DistanceFromHome"] = 4
    dataset["DistanceFromHome"] = dataset["DistanceFromHome"].astype(int)

train = train.drop(["DistanceFromHome_area"], axis=1)
combine = [train, test]
'''

'\n# 作成された範囲に沿ってラベル付け\nfor dataset in combine:\n    dataset.loc[ dataset["DistanceFromHome"] <= 2.688, "DistanceFromHome"] = 1\n    dataset.loc[(dataset["DistanceFromHome"] > 2.688) & (dataset["DistanceFromHome"] <= 9.346),\n                "DistanceFromHome"] = 2\n    dataset.loc[(dataset["DistanceFromHome"] > 9.346) & (dataset["DistanceFromHome"] <= 15.231),\n                "DistanceFromHome"]   = 3\n    dataset.loc[ dataset["DistanceFromHome"] > 15.231, "DistanceFromHome"] = 4\n    dataset["DistanceFromHome"] = dataset["DistanceFromHome"].astype(int)\n\ntrain = train.drop(["DistanceFromHome_area"], axis=1)\ncombine = [train, test]\n'

In [ ]:
for dataset in combine:
    dataset['TotalWorkingYears*JobRole'] = dataset.Department * dataset.JobRole

train.loc[:, ['TotalWorkingYears*JobRole', 'TotalWorkingYears', 'JobRole']].head(10)


,TotalWorkingYears*JobRole,TotalWorkingYears,JobRole
0,16,1,8
1,14,20,7
2,6,4,6
3,16,6,8
4,6,8,3
5,7,2,7
6,14,10,7
7,10,17,5
8,15,6,5
9,27,4,9


In [ ]:
for dataset in combine:
    dataset['EmployeeNumber*Age'] = dataset.Department * dataset.JobRole

train.loc[:, ['EmployeeNumber*Age', 'EmployeeNumber', 'Age']].head(10)


,EmployeeNumber*Age,EmployeeNumber,Age
0,16,1291,26
1,14,1587,47
2,6,1591,26
3,16,1572,46
4,6,981,25
5,7,1027,36
6,14,1420,25
7,10,635,37
8,15,1798,37
9,27,28,17


In [ ]:
# 識別のために残しておいた id を落とす
# ただし、テストデータの id は提出時の index として用いるため保持
train = train.drop(["id"], axis=1)
#test = test.drop(["id"], axis=1)
combine = [train, test]
train.shape, test.shape

((1200, 36), (800, 36))

In [ ]:
X_train = train.drop("Attrition", axis=1)
y_train = train["Attrition"]
X_test =  test.drop("id", axis=1).copy()

In [ ]:
# warning (警告を非表示、例外化する)をインポート
import warnings
warnings.filterwarnings("ignore")
# LightGBM のインポート
import lightgbm as lgb

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn import metrics

In [ ]:
X_train_lgb, X_valid_lgb, Y_train_lgb, Y_valid_lgb = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [ ]:

lgb_params = {
    "objective" : "multiclass",
    "metric" : "multi_logloss",     #"multi_logloss"
    "num_class" : 2,                # 4
    "random_state": 777,
    "max_depth" : 7,
    "learning_rate" : 0.01,
}

lgtrain = lgb.Dataset(X_train_lgb, label=Y_train_lgb)
lgtest = lgb.Dataset(X_valid_lgb, label=Y_valid_lgb)

model_lgb = lgb.train(lgb_params,
                    lgtrain,
                    num_boost_round=1000,
                    valid_sets=[lgtrain, lgtest],
                    early_stopping_rounds=100,
                    verbose_eval=100,
                   )

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000405 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1105
[LightGBM] [Info] Number of data points in the train set: 960, number of used features: 34
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Start training from score -0.189850
[LightGBM] [Info] Start training from score -1.754945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [ ]:
y_pred = model_lgb.predict(X_test).argmax(axis=1)

In [ ]:
X_test.shape

(800, 35)

In [ ]:
submit = pd.DataFrame({"index": test["id"],"Attrition": y_pred})

In [ ]:
submit.to_csv("./submit_lgb3.csv", header=False, index=False)

In [ ]:
import datetime

dt_now = datetime.datetime.now()
print(dt_now)

2023-07-08 08:53:02.099503
